In [2]:
from pyspark.sql import SparkSession

#Initialize Spark session with Iceberg configurations
spark = SparkSession.builder.appName("IcebergLocalDevelopment").config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2').config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions").config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog").config("spark.sql.catalog.local.type", "hadoop").config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg").getOrCreate()

#Verify Spark session creation
spark.sql("SHOW DATABASES").show()


25/07/13 05:54:19 WARN Utils: Your hostname, codespaces-dda585 resolves to a loopback address: 127.0.0.1; using 10.0.1.203 instead (on interface eth0)
25/07/13 05:54:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspaces/learn_iceberg/iceberg_pyspark_project/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a28b1206-ab27-42ca-a7bc-a14662fdbfb5;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.5.2/iceberg-spark-runtime-3.5_2.12-1.5.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2!iceberg-spark-runtime-3.5_2.12.jar (2981ms)
:: resolution report :: resolve 1672ms :: artifacts dl 2990ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

+---------+
|namespace|
+---------+
|  default|
+---------+



In [3]:
print(spark.conf.get("spark.sql.catalog.local"))

org.apache.iceberg.spark.SparkCatalog


In [4]:
try:
    spark.sql("CREATE NAMESPACE IF NOT EXISTS local.schema")
except Exception as e:
    print("Python Exception:", e)
    if hasattr(e, "java_exception"):
        print("Java Exception Message:", e.java_exception.getMessage())


In [5]:
print(spark.version)

3.5.1


In [6]:
!pip uninstall -y pyspark
!pip install pyspark==3.5.1


Found existing installation: pyspark 3.5.1
Uninstalling pyspark-3.5.1:
  Successfully uninstalled pyspark-3.5.1
  Using cached pyspark-3.5.1-py2.py3-none-any.whl


25/07/13 05:54:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
print(spark.version)

3.5.1


In [8]:
#Create an Iceberg table
spark.sql(""" 
		CREATE TABLE local.schema.users (
			 id INT, 
			 name STRING, 
			 age INT 
	  ) USING iceberg""")

DataFrame[]

In [9]:
#Insert some sample data
spark.sql(""" 
		INSERT INTO local.schema.users VALUES 
			(1, 'Alice', 30), 
			(2, 'Bob', 25), 
			(3, 'Charlie', 35)""")

DataFrame[]

In [10]:
#Query the data
result = spark.sql("SELECT * FROM local.schema.users") 
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+



In [12]:
spark.sql("update local.schema.users set name = 'pradeep' where id =2")

DataFrame[]

In [13]:
#Query the data
result = spark.sql("SELECT * FROM local.schema.users") 
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|pradeep| 25|
|  3|Charlie| 35|
|  1|  Alice| 30|
+---+-------+---+

